# 2. 0 Python Multithread programming

https://opensource.com/article/17/4/grok-gil

https://docs.python.org/3/library/threading.html

A thread is an entity within a process that can be scheduled for execution. Also, it is the smallest unit of processing that can be performed in an OS (Operating System).

In simple words, a thread is a sequence of such instructions within a program that can be executed independently of other code. For simplicity, you can assume that a thread is simply a subset of a process!



## Starting a Thread

In [1]:
# Python program to illustrate the concept 
# of threading 
# importing the threading module 
import threading 
  
def print_cube(num): 
    """ 
    function to print cube of given num 
    """
    print("Cube: {}".format(num * num * num)) 
  
def print_square(num): 
    """ 
    function to print square of given num 
    """
    print("Square: {}".format(num * num)) 
  
if __name__ == "__main__": 
    # creating thread 
    t1 = threading.Thread(target=print_square, args=(10,)) 
    t2 = threading.Thread(target=print_cube, args=(10,)) 
  
    # starting thread 1 
    t1.start() 
    # starting thread 2 
    t2.start() 
  
    # wait until thread 1 is completely executed 
    t1.join() 
    # wait until thread 2 is completely executed 
    t2.join() 
  
    # both threads completely executed 
    print("Done!") 

Square: 100
Cube: 1000
Done!


## Get some information of thread.
A thread contains all this information in a Thread Control Block (TCB):
Thread Identifier: Unique id (TID) is assigned to every new thread
Stack pointer: Points to thread’s stack in the process. Stack contains the local variables under thread’s scope.
Program counter: a register which stores the address of the instruction currently being executed by thread.
Thread state: can be running, ready, waiting, start or done.
Thread’s register set: registers assigned to thread for computations.
Parent process Pointer: A pointer to the Process control block (PCB) of the process that the thread lives on.
Consider the diagram below to understand the relation between process and its thread:

In [2]:
# Python program to illustrate the concept 
# of threading 
import threading 
import os 
  
def task1(): 
    print("Task 1 assigned to thread: {}".format(threading.current_thread().name)) 
    print("the thread iden: {}".format(threading.current_thread().ident)) 
    print("ID of process running task 1: {}".format(os.getpid())) 
  
def task2(): 
    print("Task 2 assigned to thread: {}".format(threading.current_thread().name)) 
    print("ID of process running task 2: {}".format(os.getpid())) 
  
if __name__ == "__main__": 
  
    # print ID of current process 
    print("ID of process running main program: {}".format(os.getpid())) 
  
    # print name of main thread 
    print("Main thread name: {}".format(threading.main_thread().name)) 
  
    # creating threads 
    t1 = threading.Thread(target=task1, name='t1') 
    t2 = threading.Thread(target=task2, name='t2')   
  
    # starting threads 
    t1.start() 
    t2.start() 
  
    # wait until all threads finish 
    t1.join() 
    t2.join() 

ID of process running main program: 43
Main thread name: MainThread
Task 1 assigned to thread: t1
the thread iden: 140118761379584
ID of process running task 1: 43
Task 2 assigned to thread: t2
ID of process running task 2: 43


## Could multithread help? 
in case...

## Problem: add 1 to n for many times.

In [1]:
import socket
n = 0

def foo():
    global n
    n += 1
#    sleep(10)
#    s = socket.socket()
#    s.connect(('python.org', 80))  # drop the GIL

In [2]:
%%time
# single thread 
for i in range(10000000):
    foo()
print(n)    

10000000
CPU times: user 1.51 s, sys: 3.95 ms, total: 1.52 s
Wall time: 1.54 s


In [30]:
%%time
#multiple threads  

threads = []

for i in range(1000):
    t = threading.Thread(target=foo)
    threads.append(t)


CPU times: user 15.7 s, sys: 332 ms, total: 16 s
Wall time: 16 s


In [26]:
%%time
for t in threads:
    t.start()

for t in threads:
    t.join()

print(n)    

11000
CPU times: user 170 ms, sys: 860 ms, total: 1.03 s
Wall time: 487 ms


which one is faster? 

what if to add a sleep(10)?

Why?

## The global interpreter lock
Here it is:

static PyThread_type_lock interpreter_lock = 0; /* This is the GIL */
This line of code is in ceval.c, in the CPython 2.7 interpreter's source code. Guido van Rossum's comment, "This is the GIL," was added in 2003, but the lock itself dates from his first multithreaded Python interpreter in 1997. On Unix systems, PyThread_type_lock is an alias for the standard C lock, mutex_t. 

Different python implements. CPython, Jython, PyPy, or IronPython
What kind of Python implementation is using?   

In [20]:
import platform
platform.python_implementation()

'CPython'

In [21]:
import dis
dis.dis(foo)

  6           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)

  8           8 LOAD_GLOBAL              1 (socket)
             10 LOAD_METHOD              1 (socket)
             12 CALL_METHOD              0
             14 STORE_FAST               0 (s)

  9          16 LOAD_FAST                0 (s)
             18 LOAD_METHOD              2 (connect)
             20 LOAD_CONST               2 (('python.org', 80))
             22 CALL_METHOD              1
             24 POP_TOP
             26 LOAD_CONST               0 (None)
             28 RETURN_VALUE


/* s.connect((host, port)) method */
static PyObject *
sock_connect(PySocketSockObject *s, PyObject *addro)
{
    sock_addr_t addrbuf;
    int addrlen;
    int res;

    /* convert (host, port) tuple to C address */
    getsockaddrarg(s, addro, SAS2SA(&addrbuf), &addrlen);

    Py_BEGIN_ALLOW_THREADS
    res = connect(s->sock_fd, addr, addrlen);
    Py_END_ALLOW_THREADS

    /* error handling and so on .... */
}

## threading conclusion
1. multiple threading is supported.
2. python code 
3. system code


In [ ]:
Questions:
    1. the more thread the faster?
    2. the relationship between cpu and thread?

## the potential issue of the multithreading solution

the result could be wrong in some cases.


In [31]:
n = 0

def foo():
    global n
    n += 1

In [32]:
>>> import dis
>>> dis.dis(foo)

  5           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


### Lock Objects
A primitive lock is a synchronization primitive that is not owned by a particular thread when locked. In Python, it is currently the lowest level synchronization primitive available, implemented directly by the _thread extension module.

with some_lock:
    # do something...
is equivalent to:

some_lock.acquire()
try:
    # do something...
finally:
    some_lock.release()

In [33]:

n = 0
locker = threading.Lock()
def foo():
    with locker:
        global n
        n += 1

In [34]:
%%time
#multiple threads  

threads = []

for i in range(10):
    t = threading.Thread(target=foo)
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print(n) 

10
CPU times: user 1.54 s, sys: 25.8 ms, total: 1.56 s
Wall time: 1.56 s


# Thread pool example
https://docs.python.org/3/library/concurrent.futures.html?highlight=concurence


In [35]:
import concurrent.futures

n = 0
locker = threading.Lock()
def foo():
    with locker:
        global n
        n += 1

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(foo, )
print(n)


0


In [8]:
concurrent.futures.ThreadPoolExecuto